<a href="https://colab.research.google.com/github/katerinneramos/data/blob/main/Analizar_Excel_con_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importación de librerías

In [ ]:
import pandas as pd
import sqlite3
import io

# Importación de archivo Excel

In [ ]:
# Cargar del dataset (*.xlsx) desde el equipo local.
from google.colab import files 
uploaded = files.upload()

Saving train.xlsx to train.xlsx


In [ ]:
# Asignar el dataset a un objeto dataframe.
train = pd.read_excel(io.BytesIO(uploaded['train.xlsx']), sheet_name = None)
        
# Listar hojas (sheets).        
train.keys()

# NOTA: buscar y reemplazar en adelante por el nombre de la hoja (sheet) que se desea consultar.

dict_keys(['DIM_VERSION', 'DIM_EXCHANGE_RATE', 'DIM_SKU', 'DIM_PRODUCT', 'DIM_PRODUCT_FAMILY', 'DIM_MOLECULE', 'DIM_TERAPHEUTIC_AREA', 'DIM_SUB_COUNTRY', 'DIM_COUNTRY', 'DIM_SUB_REGION', 'DIM_REGION', 'DIM_CHANNEL', 'DIM_MARKET', 'DIM_DIVISION'])

In [ ]:
# Mostrar datos.
train['DIM_SKU']

,sku_key,sku_code,sku_country,sku_desc,product_key
0,ALGT-209401,ALGT-209401,GUATEMALA,CLIMABEL,P171
1,ALGT-209404,ALGT-209404,GUATEMALA,DANIELE,P205
2,ALGT-209405,ALGT-209405,GUATEMALA,PLANIFERT,P648
3,ALGT-210172,ALGT-210172,GUATEMALA,OBBIAT,P602
4,ALGT-210174,ALGT-210174,GUATEMALA,OPRAH,P620
...,...,...,...,...,...
2431,ELM1-BGT22-41,ELM1-BGT22-41,MEXICO,MIDAZOLAM 7.5MG 1,P531
2432,ELM1-BGT22-42,ELM1-BGT22-42,MEXICO,MIDAZOLAM 10MG 1,P531
2433,OTHER COLOMBIA,OTHER COLOMBIA,COLOMBIA,OTHER COLOMBIA,P589
2434,OTHER ECUADOR,OTHER ECUADOR,ECUADOR,OTHER ECUADOR,P589


In [ ]:
# Mostrar información general.
train['DIM_SKU'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2436 entries, 0 to 2435
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku_key      2436 non-null   object
 1   sku_code     2436 non-null   object
 2   sku_country  2436 non-null   object
 3   sku_desc     2436 non-null   object
 4   product_key  2435 non-null   object
dtypes: object(5)
memory usage: 95.3+ KB


In [ ]:
# Mostrar información general estadística.
train['DIM_SKU'].describe()

,sku_key,sku_code,sku_country,sku_desc,product_key
count,2436,2436,2436,2436,2435
unique,2436,2436,8,2123,896
top,ALGT-209401,ALGT-209401,PARAGUAY,Dummy LTP22-12,P801
freq,1,1,1234,10,234


In [ ]:
# Contar valores nulos para cada atributo (columna).
train['DIM_SKU'].isnull().sum()

sku_key        0
sku_code       0
sku_country    0
sku_desc       0
product_key    1
dtype: int64

# Configuración del ambiente SQL

In [ ]:
# Función para pasar un df a sql db.
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    # Configurar de accesos.
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Encontrar columnas en el df.
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Conectar a una bd.
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Crear una tabla para consultar.
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Asociar el df.
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    con.commit()
    con.close()

# Función para ejecutar consultas.
def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame: 
    
    # Conectar a la bd SQL.
    con = sqlite3.connect(db_name)

    # Ejecutar la consulta SQL.
    cursor = con.execute(sql_query_string)

    # Extraer el resultado.
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    con.close()

    # Devolver df.
    return pd.DataFrame(result_data, columns=cols)

In [ ]:
# Pasar df a sql db.
pd_to_sqlDB(train['DIM_SKU'],
            table_name='DIM_SKU',
            db_name='default.db')

2022-03-18 14:11:21 INFO: SQL DB default.db created
2022-03-18 14:11:21 INFO: SQL Table DIM_SKU created with 5 columns
2022-03-18 14:11:21 INFO: 2436 rows uploaded to DIM_SKU


# Sentencias SQL

In [ ]:
# Detectar duplicados
sql_query_string = """

    SELECT *
      FROM DIM_SKU
     WHERE product_key is null

"""
 
# Ejecutar consulta SQL.
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,sku_key,sku_code,sku_country,sku_desc,product_key
0,ALGT-223699,ALGT-223699,GUATEMALA,ACERTINA 28 CAPS,None
